# Model Recovery & Export (Fixed)

Use this notebook to export your saved checkpoints from Google Drive to GGUF.

## Requirements
- Google Colab (T4 GPU)
- Google Drive mounted with `doctor_online_data/checkpoints/`

In [ ]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    !pip install --no-deps xformers trl peft accelerate bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
import torch
import os
from google.colab import drive

# Mount Drive
drive.mount('/content/drive')
BASE_PATH = "/content/drive/MyDrive/doctor_online_data/"
CHECKPOINT_DIR = f"{BASE_PATH}checkpoints/"

# 1. Find Latest Adapter
stages = ["Stage4_Followup_adapter", "Stage3_Meds_adapter", "Stage2_Domain_adapter", "Stage1_Instruction_adapter"]
adapter_path = None

if os.path.exists(CHECKPOINT_DIR):
    for stage in stages:
        path = os.path.join(CHECKPOINT_DIR, stage)
        if os.path.exists(path):
            print(f"Found latest adapter: {stage}")
            adapter_path = path
            break
else:
    print(f"Checkpoint directory not found at {CHECKPOINT_DIR}")

if not adapter_path:
    print("No adapters found in checkpoints! Please check your Drive folder.")

In [ ]:
# 2. Load Model & Adapter (Correct Way)
if adapter_path:
    print(f"Loading model from {adapter_path}...")
    # Unsloth automatically loads the base model + adapter when you point to the adapter path
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = adapter_path,
        max_seq_length = 2048,
        dtype = None,
        load_in_4bit = True,
    )
    
    # Enable native 2x faster inference (optional, but good practice)
    FastLanguageModel.for_inference(model)
else:
    print("Cannot load model: No adapter path found.")

In [ ]:
# 3. Export to GGUF
if adapter_path:
    print("Saving to GGUF (q4_k_m)...")
    try:
        # Save directly to Drive to avoid copying later
        output_path = f"{BASE_PATH}medical_llama3_recovered.gguf"
        
        # Note: save_pretrained_gguf saves to a local file first, then we move it.
        # Or we can specify a local path and then move.
        model.save_pretrained_gguf("model_recovered", tokenizer, quantization_method = "q4_k_m")
        
        # Move to Drive
        !cp model_recovered-unsloth.Q4_K_M.gguf "{output_path}"
        print(f"SUCCESS! Model saved to: {output_path}")
    except Exception as e:
        print(f"Export failed: {e}")